# Callbacks & Regularizations

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
(images, labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
images = images.reshape((60000, 28*28)).astype('float32') / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [ ]:
def get_mnist_model():
  inputs = layers.Input(shape=(28*28,))
  x = layers.Dense(256, activation='relu')(inputs)
  outputs = layers.Dense(10, activation='softmax')(x)
  model = keras.Model(inputs, outputs)
  return model

## 1- Regularizations

In [ ]:
def get_mnist_model_regularized():
  inputs = layers.Input(shape=(28*28,))
  x = layers.Dense(256, activation='relu',
                   kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(inputs)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(20, activation='relu',
                    kernel_regularizer=regularizers.L2(1e-4))(x)
  outputs = layers.Dense(10, activation='softmax')(x)
  model = keras.Model(inputs, outputs)
  return model

In [ ]:
model = get_mnist_model()
# model = get_mnist_model_regularized()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_5 (Dense)             (None, 256)               200960    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 20)                5140      
                                                                 
 dense_7 (Dense)             (None, 10)                210       
                                                                 
Total params: 206,310
Trainable params: 206,310
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 2- Callbacks

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint('checkpoint.h5',
                                                   save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                               patience=2)
logger = keras.callbacks.CSVLogger('logg.csv')
callback_list = [model_checkpoint, early_stopping, logger]

In [ ]:
model.fit(train_images, train_labels,
          epochs=10,
          validation_data=(val_images, val_labels),
          callbacks=callback_list)

Epoch 1/10
1563/1563 [==============================] - 12s 7ms/step - loss: 0.2416 - accuracy: 0.9297 - val_loss: 0.1342 - val_accuracy: 0.9586
Epoch 2/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.1020 - accuracy: 0.9699 - val_loss: 0.1031 - val_accuracy: 0.9696
Epoch 3/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.0664 - accuracy: 0.9801 - val_loss: 0.0994 - val_accuracy: 0.9696
Epoch 4/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0487 - accuracy: 0.9851 - val_loss: 0.0773 - val_accuracy: 0.9781
Epoch 5/10
1563/1563 [==============================] - 10s 7ms/step - loss: 0.0329 - accuracy: 0.9900 - val_loss: 0.0805 - val_accuracy: 0.9764
Epoch 6/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.0253 - accuracy: 0.9919 - val_loss: 0.0782 - val_accuracy: 0.9767
